In [2]:
# Import required libraries
import pandas as pd
import dash
import dash_html_components as html
import dash_core_components as dcc
from dash.dependencies import Input, Output
import plotly.express as px

# Read the airline data into pandas dataframe
spacex_df = pd.read_csv("spacex_launch_dash.csv")
max_payload = spacex_df['Payload Mass (kg)'].max()
min_payload = spacex_df['Payload Mass (kg)'].min()
launch_sites = spacex_df['Launch Site'].unique()
# Create a dash application
app = dash.Dash(__name__)

# Create an app layout
app.layout = html.Div(children=[html.H1('SpaceX Launch Records Dashboard',
                                        style={'textAlign': 'center', 'color': '#503D36',
                                               'font-size': 40}),
                                # TASK 1: Add a dropdown list to enable Launch Site selection
                                # The default select value is for ALL sites
                                dcc.Dropdown(id='site-dropdown',
                                options=[{'label': 'All', 'value': 'All'}] + [{'label': site, 'value': site} for site in launch_sites]
,
                                value='All',
                                placeholder="Select a Launch Site here",
                                searchable=True
                                ),
                                html.Br(),

                                # TASK 2: Add a pie chart to show the total successful launches count for all sites
                                # If a specific launch site was selected, show the Success vs. Failed counts for the site
                                html.Div(dcc.Graph(id='success-pie-chart')),
                                html.Br(),

                                html.P("Payload range (Kg):"),
                                # TASK 3: Add a slider to select payload range
                                dcc.RangeSlider(id='payload-slider',
                                min=0, max=10000, step=1000,
                                marks={0: '0',
                                    2500: '2500',
                                    5000: '5000',
                                    7500: '7500',
                                    },
                                value=[0, 2500,5000,7500]),

                                # TASK 4: Add a scatter chart to show the correlation between payload and launch success
                                html.Div(dcc.Graph(id='success-payload-scatter-chart')),
                                ])

# TASK 2:
# Function decorator to specify function input and output
@app.callback(Output(component_id='success-pie-chart', component_property='figure'),
              Input(component_id='site-dropdown', component_property='value'))
def get_pie_chart(entered_site):
    filtered_df = spacex_df
    if entered_site == 'All':
        fig = px.pie(filtered_df, values='class', 
        names='Launch Site', 
        title='Launch Sites Distribution')
        return fig
    else:
        filtered_df = spacex_df[spacex_df['Launch Site'] == entered_site]
        title = f"Success Rate for {entered_site}"

        success_counts = filtered_df['class'].value_counts()
        print(success_counts)
        labels = ['Failure', 'Success']
        fig = px.pie(values=success_counts, names=labels, title=title)
        return fig
        # return the outcomes piechart for a selected site

# TASK 4:
# Add a callback function for `site-dropdown` and `payload-slider` as inputs, `success-payload-scatter-chart` as output
@app.callback(Output(component_id='success-payload-scatter-chart', component_property='figure'),
              Input(component_id='site-dropdown', component_property='value'),
              Input(component_id="payload-slider", component_property="value"))
def get_scatter_chart(entered_site, payload_range):
    if entered_site == 'All':
        filtered_df = spacex_df.copy()
        title = 'Payload vs. Outcome (All Sites)'
        color_column = 'Booster Version Category'
    else:
        filtered_df = spacex_df[spacex_df['Launch Site'] == entered_site].copy()
        title = f"Payload vs. Outcome ({entered_site})"
        color_column = 'Booster Version Category'

    # Apply payload range filter
    if payload_range:
        filtered_df = filtered_df[(filtered_df['Payload Mass (kg)'] >= payload_range[0]) & 
                                  (filtered_df['Payload Mass (kg)'] <= payload_range[1])]

    # Create scatter plot
    fig = px.scatter(filtered_df, x='Payload Mass (kg)', y='class', color=color_column,
                     title=title, labels={'Payload Mass (kg)': 'Payload Mass (kg)', 'class': 'Outcome'})

    return fig

# Run the app
if __name__ == '__main__':
    app.run_server()


Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [10/Jul/2023 19:43:40] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [10/Jul/2023 19:43:40] "GET /_dash-component-suites/dash/deps/polyfill@7.v2_9_3m1684838536.12.1.min.js HTTP/1.1" 200 -
127.0.0.1 - - [10/Jul/2023 19:43:40] "GET /_dash-component-suites/dash/deps/react-dom@16.v2_9_3m1684838536.14.0.min.js HTTP/1.1" 200 -
127.0.0.1 - - [10/Jul/2023 19:43:40] "GET /_dash-component-suites/dash/deps/react@16.v2_9_3m1684838536.14.0.min.js HTTP/1.1" 200 -
127.0.0.1 - - [10/Jul/2023 19:43:40] "GET /_dash-component-suites/dash/deps/prop-types@15.v2_9_3m1684838536.8.1.min.js HTTP/1.1" 200 -
127.0.0.1 - - [10/Jul/2023 19:43:40] "GET /_dash-component-suites/dash/dash-renderer/build/dash_renderer.v2_9_3m1684838536.min.js HTTP/1.1" 200 -
127.0.0.1 - - [10/Jul/2023 19:43:40] "GET /_dash-component-suites/dash/dcc/dash_core_components.v2_9_2m1684838536.js HTTP/1.1" 200 -
127.0.0.1 - - [10/Jul/2023 19:43:40] "GET /_dash-component-suites/d

0    6
1    4
Name: class, dtype: int64


127.0.0.1 - - [10/Jul/2023 19:43:48] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [10/Jul/2023 19:43:48] "POST /_dash-update-component HTTP/1.1" 200 -


0    19
1     7
Name: class, dtype: int64


127.0.0.1 - - [10/Jul/2023 19:43:50] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [10/Jul/2023 19:43:50] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [10/Jul/2023 19:43:54] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [10/Jul/2023 19:43:58] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [10/Jul/2023 19:44:01] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [10/Jul/2023 19:44:05] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [10/Jul/2023 19:44:07] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [10/Jul/2023 19:44:10] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [10/Jul/2023 19:44:12] "POST /_dash-update-component HTTP/1.1" 200 -
